<a href="https://colab.research.google.com/github/ikonthomas/TM-2-Portfolio/blob/master/Data_Munging_Wikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

# Data Munging

## Relational Data


Here I will explore a bit of data and see how combining different sets of data can help  generate useful features.

First we need some data.  We will make use of some data from Wikipedia and we will use the pandas `read_html` function to scrape the data from a particular webpage.  We will study the top 10 companies in the Fortune Global 500 which conveniently have their own Wikipedia page.

We will download the data in tabular form, but work with it as a list of dictionaries, as most data in the real world is  a bit messy and unstructured.

In [0]:
import pandas as pd
import json
df = pd.read_html('https://en.wikipedia.org/wiki/Fortune_Global_500', header=0)[0]
fortune_500 = json.loads(df.to_json(orient='records'))
df

,Rank,Company,Country,Industry,Revenue in USD
0,1,Walmart,United States,Retail,$500 billion
1,2,State Grid,China,Power,$349 billion
2,3,Sinopec Group,China,Petroleum,$327 billion
3,4,China National Petroleum,China,Petroleum,$326 billion
4,5,Royal Dutch Shell,Netherlands United Kingdom,Petroleum,$312 billion
5,6,Toyota Motor,Japan,Automobiles,$265 billion
6,7,Volkswagen,Germany,Automobiles,$260 billion
7,8,BP,United Kingdom,Petroleum,$245 billion
8,9,Exxon Mobil,United States,Petroleum,$244 billion
9,10,Berkshire Hathaway,United States,Holding,$242 billion


Lets look at the data.

In [0]:
fortune_500

[{u'Company': u'Walmart',
  u'Country': u'United States',
  u'Industry': u'Retail',
  u'Rank': 1,
  u'Revenue in USD': u'$500 billion'},
 {u'Company': u'State Grid',
  u'Country': u'China',
  u'Industry': u'Power',
  u'Rank': 2,
  u'Revenue in USD': u'$349 billion'},
 {u'Company': u'Sinopec Group',
  u'Country': u'China',
  u'Industry': u'Petroleum',
  u'Rank': 3,
  u'Revenue in USD': u'$327 billion'},
 {u'Company': u'China National Petroleum',
  u'Country': u'China',
  u'Industry': u'Petroleum',
  u'Rank': 4,
  u'Revenue in USD': u'$326 billion'},
 {u'Company': u'Royal Dutch Shell',
  u'Country': u'Netherlands \xa0United Kingdom',
  u'Industry': u'Petroleum',
  u'Rank': 5,
  u'Revenue in USD': u'$312 billion'},
 {u'Company': u'Toyota Motor',
  u'Country': u'Japan',
  u'Industry': u'Automobiles',
  u'Rank': 6,
  u'Revenue in USD': u'$265 billion'},
 {u'Company': u'Volkswagen',
  u'Country': u'Germany',
  u'Industry': u'Automobiles',
  u'Rank': 7,
  u'Revenue in USD': u'$260 billion'},


There really isn't that much information here, we will need to bring in additional data sources to get any further understanding of these companies.

The first question we might want to ask is how many employees does it take to get that revenue, in other words, what is the revenue per employee?  Luckily, we can use Wikipedia to get that data as well, I have scraped this data manually (all from Wikipedia) and created the following dictionary.

In [0]:
other_data = [
    {"name": "Walmart",
     "employees": 2300000,
     "year founded": 1962
    },
    {"name": "State Grid Corporation of China",
     "employees": 927839,
     "year founded": 2002},
    {"name": "China Petrochemical Corporation",
     "employees":358571,
     "year founded": 1998
     },
    {"name": "China National Petroleum Corporation",
     "employees": 1636532,
     "year founded": 1988},
    {"name": "Toyota Motor Corporation",
     "employees": 364445,
     "year founded": 1937},
    {"name": "Volkswagen AG",
     "employees": 642292,
     "year founded": 1937},
    {"name": "Royal Dutch Shell",
     "employees": 92000,
     "year founded": 1907},
    {"name": "Berkshire Hathaway Inc.",
     "employees":377000,
     "year founded": 1839},
    {"name": "BP",
     "employees": 123000,
     "year founded": 1976},
    {"name": "Exxon Mobile Corporation",
     "employees": 69600,
     "year founded": 1999}
]

Some data have a slightly different name than in our original set, so we will keep a dictionary of mappings between the two.  We only include the mapping in the dictionary if there is a difference.

In [0]:
mapping = {"China Petrochemical Corporation":"Sinopec Group",
           "State Grid Corporation of China":"State Grid",
           "China National Petroleum Corporation":"China National Petroleum",
           "Toyota Motor Corporation": "Toyota Motor",
           "Volkswagen AG":"Volkswagen",
           "Berkshire Hathaway Inc.":"Berkshire Hathaway",
           "Exxon Mobile Corporation": "Exxon Mobil"
          }

This data is one to one, meaning the data contained in one source only aligns with a single element in the other source, thus we should be able to put these together.  However, we know that the data isn't in a great form to be joined at the moment.  This is for two reasons

1. All the names will not align (we need to use our mapping)
2. The `list` structure is not optimized for looking through elements. 

While for 10 elements the second reason won't really matter, for larger data sets such performance considerations are extremely important.  We can turn this list of dictionaries into a dictionary of dictionaries, so we can quickly access each element of the data.

In [0]:
dict_data = {k["name"] : k for k in other_data}
dict(list(dict_data.items())[0:3])

{'Exxon Mobile Corporation': {'employees': 69600,
  'name': 'Exxon Mobile Corporation',
  'year founded': 1999},
 'State Grid Corporation of China': {'employees': 927839,
  'name': 'State Grid Corporation of China',
  'year founded': 2002},
 'Toyota Motor Corporation': {'employees': 364445,
  'name': 'Toyota Motor Corporation',
  'year founded': 1937}}

Now we can easily compute the revenue per employee, we need to map the "Company" value in our original data with the "name" column of this other data, but we also need to use the mapping to ensure the columns will line up.  We in general don't want to mutate our original data, so lets make a new list of dictionaries with this new feature (revenue per employee).  On the course of doing this, we will need to handle converting some numbers like `$500 Billion` to a numeric value.  Lets create a function to do this.

In [0]:
def convert_revenue(x):
    return float(x.lstrip('$').rstrip('billion')) * 1e9

assert convert_revenue('$500 billion') == 500e9

Now we can convert this revenue, but unfortunately the mapping we have written goes the wrong way, lets invert it.

In [0]:
inv_map = {v : k for k,v in mapping.iteritems()}

Now we should be able to create a few functions to compute this revenue per employee and create a data list.

In [0]:
def rev_per_emp(company):
    name = company[u'Company']
    n_employees = dict_data[inv_map.get(name, name)].get('employees')
    company['rev per emp'] = convert_revenue(company[u'Revenue in USD'])/n_employees
    return company

def compute_copy(d, func):
    return func({k:v for k,v in d.iteritems()})

data = map(lambda x : compute_copy(x, rev_per_emp), fortune_500)

Lets take a look at our new data and also the old data to ensure we didn't mutate anything.

In [0]:
data[:2]

[{u'Company': u'Walmart',
  u'Country': u'United States',
  u'Industry': u'Retail',
  u'Rank': 1,
  u'Revenue in USD': u'$500 billion',
  'rev per emp': 217391.30434782608},
 {u'Company': u'State Grid',
  u'Country': u'China',
  u'Industry': u'Power',
  u'Rank': 2,
  u'Revenue in USD': u'$349 billion',
  'rev per emp': 376142.84374767606}]

In [0]:
fortune_500[:2]

[{u'Company': u'Walmart',
  u'Country': u'United States',
  u'Industry': u'Retail',
  u'Rank': 1,
  u'Revenue in USD': u'$500 billion'},
 {u'Company': u'State Grid',
  u'Country': u'China',
  u'Industry': u'Power',
  u'Rank': 2,
  u'Revenue in USD': u'$349 billion'}]

Now we can sort these values.  We first can select out on the elements we care about and then sort that list.

In [0]:
rev_per_emp = sorted([(i[u'Company'], i['rev per emp']) for i in data], 
                   key=lambda x : x[1],
                   reverse=True)
rev_per_emp

TypeError: ignored



Now lets pull in some other data (as this is data science, more data is always better!).  We can see that these companies are in a few different industries, lets find out which ones.

In [0]:
from collections import Counter
Counter(i[u'Industry'] for i in data)

One thing we might want to know is what sort of market share they have of the specific industry to which they belong.  Let's look at the two industries that categorize the 6 of the top 10, `Automobiles` and `Petroleum`.  We can select only those elements of our data to work with.

In [0]:
sub_data = [i for i in data if i[u'Industry'] in [u'Automobiles', u'Petroleum']]
sub_data

It might be the case that the each particular category has a different relevant metric for market share.  For example, we could look at total revenue for a car company or we could look at cars produced.  

So for the automobile industry we will look at the percent total of cars produced.  We can get this data again from Wikipedia.

In [0]:
df_list = pd.read_html("https://en.wikipedia.org/wiki/Automotive_industry", header=0)
car_totals = json.loads(df_list[0].to_json(orient="records"))
car_by_man = json.loads(df_list[2].to_json(orient='records'))

In [0]:
car_totals[:2]

[{u'Change': u'\u2014',
  u'Production': 54434000,
  u'Source': u'[17]',
  u'Year': 1997},
 {u'Change': u'2.7%',
  u'Production': 52987000,
  u'Source': u'[17]',
  u'Year': 1998}]

In [0]:
car_by_man[:2]

[{u'Country': u'Japan',
  u'Group': u'Toyota',
  u'Rank': 1,
  u'Vehicles': 10213486},
 {u'Country': u'Germany',
  u'Group': u'Volkswagen Group',
  u'Rank': 2,
  u'Vehicles': 10126281}]

Now lets get only the groups we care about and divide by the total production which we will take as the latest year.

In [0]:
total_prod = sorted((i[u"Year"], i[u'Production']) for i in car_totals)[-1][1]
total_prod

97302534

Now we can find the market share for each of the car companies. We will keep track of a market share dictionary.  We will again need to keep track of some slight name differences.

In [0]:
car_by_man_dict = {i[u'Group']:i[u'Vehicles'] for i in car_by_man}
market_share = {}
for name, orig_name in zip(['Toyota', 'Volkswagen Group'], ['Toyota', 'Volkswagen']):
    market_share[orig_name] = car_by_man_dict[name]/ float(total_prod)
    
market_share

{'Toyota': 0.1049662899837737, 'Volkswagen': 0.10407006460900597}

Now we can do the same for the Petroleum industry, but in this case, lets compute the market share by revenue.  On Wikipedia, we can find a list of oil companies by revenue.  Although its not a complete list, it has enough companies that we don't expect the companies left off the list to contribute greatly to our analysis.

In [0]:
rev = pd.read_html("https://en.wikipedia.org/wiki/List_of_largest_oil_and_gas_companies_by_revenue", header=1)[0]
rev = rev.iloc[:, 1:3]
rev.columns = ['Company', 'Revenue']
oil_data = json.loads(rev.to_json(orient="records"))
oil_data[:2]

[{u'Company': u'Saudi Aramco', u'Revenue': u'465.49'},
 {u'Company': u'Sinopec Group', u'Revenue': u'448.00'}]

Now we can compute the totals and market share.  Since the data here might be slightly different (perhaps older) than our original data, we will compute the market share of each company within this data set, then pull out the numbers we care about.

In [0]:
total = sum([float(i[u'Revenue'].rstrip('*')) for i in oil_data])
shares = {i[u'Company']:float(i[u'Revenue'].rstrip('*'))/total for i in oil_data}
print total

5532.09


Now we can pull out the companies we care about in the petroleum industry.

In [0]:
petro_companies = [i[u'Company'] for i in data if i['Industry'] == u'Petroleum']
petro_companies

Lets check if these are all in the our shares dictionary.

In [0]:
[(i, i in shares) for i in petro_companies]

Some of these companies are directly there, and looking through our dictionary, we can see the others are there without exact names.

In [0]:
shares.keys()

So lets make a fuzzy match, this will be a pretty simple one where it will try to match words in a name and take the maximum number of matches.

In [0]:
def fuzzy_match(word, s):
    words = set(word.split(' '))
    overlaps = [(k, len(v.intersection(words))) for k, v in s.iteritems()]
    return max(overlaps, key=lambda x : x[1])[0]

In [0]:
split_names = {i: set(i.split(' ')) for i in shares.keys()}
for i in petro_companies:
    match = fuzzy_match(i, split_names)
    print "matched {} to {}".format(i, match)
    market_share[i] = shares[match]

NameError: ignored

In [0]:
market_share

## by industry
We have some nice examples of data munging, now lets see an example of keeping data in a relational fashion.  Lets say we want to add another feature which is the growth of each industry.  If we were to store this data as a single quantity, we would be saving a bunch of extra information, we would be much better off extracting this information and keeping it in a single table so we are not replicating by industry.

## The more common use case With Pandas

Now we can also perform these same computations with Pandas, lets see how this compares.

In [0]:
df = pd.read_html('https://en.wikipedia.org/wiki/Fortune_Global_500', header=0)[0]
df

,Rank,Company,Country,Industry,Revenue in USD
0,1,Walmart,United States,Retail,$500 billion
1,2,State Grid,China,Power,$349 billion
2,3,Sinopec Group,China,Petroleum,$327 billion
3,4,China National Petroleum,China,Petroleum,$326 billion
4,5,Royal Dutch Shell,Netherlands United Kingdom,Petroleum,$312 billion
5,6,Toyota Motor,Japan,Automobiles,$265 billion
6,7,Volkswagen,Germany,Automobiles,$260 billion
7,8,BP,United Kingdom,Petroleum,$245 billion
8,9,Exxon Mobil,United States,Petroleum,$244 billion
9,10,Berkshire Hathaway,United States,Holding,$242 billion


In [0]:
df['rev'] = df['Revenue in USD'].apply(convert_revenue)
df['employees'] = df['Company'].apply(lambda x : dict_data[inv_map.get(x, x)].get('employees'))
df['rev_per_employee'] = df['rev'] / df['employees'].astype(float)
df.sort_values(by='rev_per_employee', ascending=False)

,Rank,Company,Country,Industry,Revenue in USD,rev,employees,rev_per_employee
8,9,Exxon Mobil,United States,Petroleum,$244 billion,2.440000e+11,69600,3.505747e+06
4,5,Royal Dutch Shell,Netherlands United Kingdom,Petroleum,$312 billion,3.120000e+11,92000,3.391304e+06
7,8,BP,United Kingdom,Petroleum,$245 billion,2.450000e+11,123000,1.991870e+06
2,3,Sinopec Group,China,Petroleum,$327 billion,3.270000e+11,358571,9.119533e+05
5,6,Toyota Motor,Japan,Automobiles,$265 billion,2.650000e+11,364445,7.271330e+05
9,10,Berkshire Hathaway,United States,Holding,$242 billion,2.420000e+11,377000,6.419098e+05
6,7,Volkswagen,Germany,Automobiles,$260 billion,2.600000e+11,642292,4.048003e+05
1,2,State Grid,China,Power,$349 billion,3.490000e+11,927839,3.761428e+05
0,1,Walmart,United States,Retail,$500 billion,5.000000e+11,2300000,2.173913e+05
3,4,China National Petroleum,China,Petroleum,$326 billion,3.260000e+11,1636532,1.992017e+05


In [0]:
df_list = pd.read_html("https://en.wikipedia.org/wiki/Automotive_industry", header=0)
df_totals = df_list[0]
df_by_man = df_list[2]
df_totals


,Year,Production,Change,Source
0,1997,54434000,—,[17]
1,1998,52987000,2.7%,[17]
2,1999,56258892,6.2%,[18]
3,2000,58374162,3.8%,[19]
4,2001,56304925,3.5%,[20]
5,2002,58994318,4.8%,[21]
6,2003,60663225,2.8%,[22]
7,2004,64496220,6.3%,[23]
8,2005,66482439,3.1%,[24]
9,2006,69222975,4.1%,[25]


In [0]:
total_prod = df_totals.sort_values(by='Year').iloc[-1]['Production']
total_prod

97302534

In [0]:
df_by_man['share'] = df_by_man['Vehicles'].apply(lambda x : float(x)/ total_prod)
market_share = df_by_man.set_index('Group')['share'][['Toyota', 'Volkswagen Group']]
market_share

Group
Toyota              0.104966
Volkswagen Group    0.104070
Name: share, dtype: float64

In [0]:
rev = pd.read_html("https://en.wikipedia.org/wiki/List_of_largest_oil_and_gas_companies_by_revenue", header=1)[0]
rev = rev.iloc[:, 1:3]
rev.columns = ['Company', 'Revenue']
rev

,Company,Revenue
0,Saudi Aramco,465.49
1,Sinopec Group,448.00
2,China National Petroleum Corporation,428.62
3,Exxon Mobil,268.9
4,Royal Dutch Shell,265
5,Kuwait Petroleum Corporation,251.94
6,BP,222.8
7,Total SA,212
8,Lukoil,144.17
9,Eni,131.82


In [0]:
rev['rev_clean'] = rev['Revenue'].apply(lambda x : float(x.rstrip('*')))
total = rev['rev_clean'].sum()
total


5532.089999999999

In [0]:
rev['share'] = rev['rev_clean'] / total
rev

,Company,Revenue,rev_clean,share
0,Saudi Aramco,465.49,465.49,0.084144
1,Sinopec Group,448.00,448.00,0.080982
2,China National Petroleum Corporation,428.62,428.62,0.077479
3,Exxon Mobil,268.9,268.90,0.048607
4,Royal Dutch Shell,265,265.00,0.047902
5,Kuwait Petroleum Corporation,251.94,251.94,0.045542
6,BP,222.8,222.80,0.040274
7,Total SA,212,212.00,0.038322
8,Lukoil,144.17,144.17,0.026061
9,Eni,131.82,131.82,0.023828


In [0]:
rev = rev[rev['Company'].isin(['Exxon Mobil', 'Sinopec', 'China National Petroleum Corporation', 'Royal Dutch Shell'])].copy()
rev

,Company,Revenue,rev_clean,share
2,China National Petroleum Corporation,428.62,428.62,0.077479
3,Exxon Mobil,268.9,268.90,0.048607
4,Royal Dutch Shell,265,265.00,0.047902


In [0]:
# do fuzzy search
split_names = {i: set(i.split(' ')) for i in df['Company']}

def fuzzy(word):
    return fuzzy_match(word, split_names)

rev['name'] = rev['Company'].apply(fuzzy)
rev

,Company,Revenue,rev_clean,share,name
2,China National Petroleum Corporation,428.62,428.62,0.077479,China National Petroleum
3,Exxon Mobil,268.9,268.90,0.048607,Exxon Mobil
4,Royal Dutch Shell,265,265.00,0.047902,Royal Dutch Shell


In [0]:
ms2 = df.merge(rev[['share', 'name']], left_on='Company', right_on='name')

Now we want to put these together and get only the company and the market share.

In [0]:
ms = market_share.reset_index()[['Group','share']]
ms.columns = ['Company', 'share']
pd.concat([ms, ms2[['Company', 'share']]])

,Company,share
0,Toyota,0.104966
1,Volkswagen Group,0.104070
0,China National Petroleum,0.077479
1,Royal Dutch Shell,0.047902
2,Exxon Mobil,0.048607
